<a href="https://colab.research.google.com/github/harveyrutland/swarm_routines/blob/main/swarm_boatproperties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class SetEnv:
  def __init__(self):
    self.rect_startx = random.uniform(2.5, 5)
    rect_starty1 = random.uniform(0, 0.5)
    rect_starty2 = random.uniform(3.5, 4)
    randomy = random.uniform(0,1)
    if randomy == 0:
      self.rect_starty = rect_starty1
    else:
      self.rect_starty = rect_starty2

  def print_boats(self):
        print(self.num)
  
  def clear_frames(self):
    #frame of animation
    ax.clear()
    ax.set_xlim(( 0, 20))
    ax.set_ylim((0,6))
  
  def add_start_box(self):
    #start square 
    ax.add_patch(patches.Rectangle((self.rect_startx, self.rect_starty),3,3,fill=False))
    return self.rect_startx, self.rect_starty
    #frames
    # p = plt.Polygon(np.c_[x,y], color='white')
    # ax.add_patch(p)





#######
class AddBoats(SetEnv):
   def __init__(self, set_env):
        self.ascending = True
        self.seen_boat1 = False
        self.search_state = True 


        # boat view point and and angle variables 
        self.angle = random.uniform(0, 360)  
        # self.angle = math.pi + math.pi/2

        # random angle view point rotation limits are set with -1.5 to 1.5 rad as upper and lower limits self.angle is subtracted to adjust to position of boat 
        self.boat_rotation = self.angle



        # self.rotation = math.radians(self.angle) 
        self.rotation = self.angle
        #sets the correct boat starting position relative to 0 
        self.boat_rotation_offset = 0
        self.perp_offsetdjustment = 0 
        self.dist = 0 
        

        #---- variable to dtermine direction in which shark is travelling ----# 
        self.last_angle = 0
        self.first_angle = 0 


        self.follow_shark_perp = False
        self.direction_check_interval = 0


        #change back to start random
        self.boat1_startx = random.uniform(set_env.rect_startx, set_env.rect_startx+1)
        self.boat1_starty = random.uniform(set_env.rect_starty, set_env.rect_starty+1)

  
   def set_rotation(self, x_fin, y_fin):

     if self.search_state == True:
       if self.ascending == True:
         self.rotation += 0.2
         if self.rotation >= (1.5 + self.angle):
           self.ascending = False
        
       if self.ascending == False:
         self.rotation += -0.2
         if self.rotation <= (-1.5 + self.angle):
           self.ascending = True 

     elif self.search_state == False:
        self.rotation = get_angle([x_fin, y_fin], [self.boat1_startx, self.boat1_starty])

     self.xoffset1 = math.cos(self.rotation)*5
     self.yoffset1 = math.sin(self.rotation)*5
     self.xoffsetL = math.cos(self.rotation - 0.5)*6
     self.yoffsetL = math.sin(self.rotation - 0.5)*6
     self.xoffsetR = math.cos(self.rotation + 0.5)*6
     self.yoffsetR = math.sin(self.rotation + 0.5)*6

     

     self.xoffset2 = math.cos(self.boat_rotation + self.perp_offsetdjustment + math.radians(self.boat_rotation_offset))*0.5
     self.yoffset2 = math.sin(self.boat_rotation + self.perp_offsetdjustment + math.radians(self.boat_rotation_offset))*0.5
   
  

   def check_search_state(self, x_fin, y_fin):

     if self.seen_boat1 == True:
        self.first_angle = get_angle([x_fin, y_fin], [self.boat1_startx, self.boat1_starty])
        # print('first anf' , self.first_angle)
        if abs(degrees(self.first_angle)) > 0:
          # print('angle between fin and boat', degrees(self.first_angle))
          self.search_state = False


   def follow_shark(self, x_fin, y_fin):
 
      if self.seen_boat1 == True:
        self.first_angle = get_angle([x_fin, y_fin], [self.boat1_startx, self.boat1_starty])
        # print('first anf' , self.first_angle)


        if abs(degrees(self.first_angle)) > 70 :
          # print('angle between fin and boat', degrees(self.first_angle))
          self.search_state = False
          self.dist = return_dist(self.boat1_startx, self.boat1_starty, x_fin[0], y_fin[0])

    

          
          if self.dist >= 2 and self.follow_shark_perp == False:
            # self.boat_rotation = self.rotation
            # self.boat1_startx, self.boat1_starty = move_to_shark(self.boat1_startx, self.boat1_starty, x_fin[0], y_fin[0])
            self.boat_rotation = self.rotation
            boat_path = np.linspace([self.boat1_startx, self.boat1_starty],[x_fin[0], y_fin[0]], 10)
            self.boat1_startx, self.boat1_starty = (boat_path[1][0], boat_path[1][1])
          
          if self.dist < 2 and self.follow_shark_perp == False:
            # write_to_log('Distance between boat and shark is:', dist)
            self.direction = 0
            self.direction_check_interval +=1
            
            if self.first_angle > self.last_angle:
              self.direction = 0
            elif self.first_angle < self.last_angle:
              self.direction = 1
            self.last_angle = self.first_angle 
            if self.direction_check_interval == 5:
              angle_changes = [90, -90]
              self.boat_rotation_offset = angle_changes[self.direction]
       
              self.follow_shark_perp = True
              # write_to_log('following perpendicular to shark')
              self.direction_check_interval = 0
     
      else:
        self.search_state = True 
        

      #follow shark perpendicular  
      if self.follow_shark_perp == True:

        self.direction_check_interval += 1
        boat_deltaY = 0
        if self.direction_check_interval == 1:

          self.boat_deltaX = adjust_position_perpX([self.boat1_startx, self.boat1_starty],[x_fin, y_fin], self.dist)
          
          
          init_angle = self.boat_rotation + self.perp_offsetdjustment + math.radians(self.boat_rotation_offset)
          radjutsment = adjust_position_perpY(self.boat_deltaX , init_angle)

          # if self.dist < 0.5:
          #   print('within safety threshold limit')
          #   self.perp_offsetdjustment = math.radians(45)
          #   boat_deltaY = self.boat_deltaX * math.tan(radians(-45))
         

          if self.dist < 2 :
            # print('less than theshold')
            self.perp_offsetdjustment = math.radians(20)
            boat_deltaY = self.boat_deltaX * math.tan(radians(-20))
          
          if self.dist > 2.5:
            # print('greater than theshold')
            self.perp_offsetdjustment = math.radians(-20)
            boat_deltaY = self.boat_deltaX * math.tan(radians(20))
            
          elif self.dist <= 2.5 and self.dist >= 2:
            self.perp_offsetdjustment = 0 

          self.boat1_startx = self.boat1_startx + abs(self.boat_deltaX)
          self.boat1_starty = self.boat1_starty + boat_deltaY - radjutsment
          self.direction_check_interval = 0

      

  
      
  
   def follow_leader(self, leaderx, leadery):
     print(leaderx, leadery)
     print('following the leader')
    #  self.boat1_startx, self.boat1_starty = move_to_leader(self.boat1_startx, self.boat1_starty, leaderx, leadery)
     self.dist = return_dist(self.boat1_startx, self.boat1_starty, leaderx, leadery)

     if self.dist >= 2 and self.follow_shark_perp == False:
  
        self.boat1_startx, self.boat1_starty = move_to_leader(self.boat1_startx, self.boat1_starty, leaderx, leadery)

          

        
   
  
   def plot_boats(self, x_fin, y_fin, boatnumber):
      ax.plot([self.boat1_startx, self.boat1_startx + self.xoffsetL], [self.boat1_starty, self.boat1_starty + self.yoffsetL] , 'bo', linestyle="--", markevery=[0], markersize=2)
      ax.plot([self.boat1_startx, self.boat1_startx + self.xoffsetR], [self.boat1_starty, self.boat1_starty + self.yoffsetR] , 'bo', linestyle="--", markevery=[0], markersize=2)
      x = np.append([self.boat1_startx, self.boat1_startx + self.xoffsetL], [self.boat1_startx, self.boat1_startx + self.xoffsetR][::-1])
      y = np.append([self.boat1_starty, self.boat1_starty + self.yoffsetL],[self.boat1_starty, self.boat1_starty + self.yoffsetR] [::-1])



      p = plt.Polygon(np.c_[x,y], color="lemonchiffon")
      ax.add_patch(p)
      view_region = Polygon([(self.boat1_startx, self.boat1_starty), (self.boat1_startx + self.xoffsetL, self.boat1_starty + self.yoffsetL),(self.boat1_startx + self.xoffsetR, self.boat1_starty + self.yoffsetR)])
      point = Point(x_fin, y_fin)

      self.seen_boat1 = view_region.contains(point)
      ax.plot([self.boat1_startx, self.boat1_startx + self.xoffset2], [self.boat1_starty, self.boat1_starty + self.yoffset2] , 'go', linestyle="-", markevery=[0], markersize=2, linewidth=2)


      ax.plot([self.boat1_startx, self.boat1_startx + self.xoffset1],
              [self.boat1_starty, self.boat1_starty + self.yoffset1] , 
              'ro', linestyle="--", 
              markevery=[0],marker=(3, 0, (35 + math.degrees(self.boat_rotation + self.perp_offsetdjustment)+ self.boat_rotation_offset)),  markersize=10)
      return self.seen_boat1, boatnumber, [self.boat1_startx, self.boat1_starty]